### A3.1.3. Branch Prediction

$$
\text{Penalty}_{\text{branch}} = (1 - \text{accuracy}) \times \text{pipeline\_depth}
$$

where accuracy is the prediction hit rate and pipeline depth is the number of stages flushed on a misprediction.

**Explanation:**

A **branch predictor** guesses the outcome of conditional branches before execution completes, allowing the pipeline to speculatively fetch and decode instructions along the predicted path. A misprediction flushes the pipeline, wasting ~15–20 cycles on modern CPUs.

**Predictor Types:**

| Predictor | Mechanism | Accuracy |
|-----------|-----------|----------|
| Static | Always predict taken (backward) / not-taken (forward) | ~60–70% |
| 1-bit | Remember last outcome per branch | ~85% |
| 2-bit saturating counter | 4 states (strongly/weakly taken/not-taken) | ~90% |
| Correlating (gshare) | XOR global history with branch PC to index counters | ~95% |
| TAGE | Tagged geometric history lengths — multiple tables with different history depths | ~97%+ |

**Branch Target Buffer (BTB):**

Caches the target address of taken branches, so the frontend can redirect fetch without waiting for decode.

**Performance Impact:**

- **Predictable patterns** (loop back-edges, monotonic conditions) are predicted with >99% accuracy.
- **Data-dependent branches** (random data, hash lookups) cause frequent mispredictions.
- **Branchless code** (conditional moves, arithmetic masks) eliminates the branch entirely.

**Example:**

Sorting an array before branching on its values makes the branch pattern monotonic (all false, then all true), increasing prediction accuracy dramatically compared to branching on unsorted random data.

In [ ]:
import numpy as np
import time

SIZE = 1_000_000
THRESHOLD = 128
data = np.random.randint(0, 256, size=SIZE)

unsorted_data = data.copy()
start = time.perf_counter()
unsorted_sum = 0
for value in unsorted_data:
    if value >= THRESHOLD:
        unsorted_sum += value
unsorted_time = time.perf_counter() - start

sorted_data = np.sort(data)
start = time.perf_counter()
sorted_sum = 0
for value in sorted_data:
    if value >= THRESHOLD:
        sorted_sum += value
sorted_time = time.perf_counter() - start

print(f"Sum (unsorted): {unsorted_sum}, time: {unsorted_time:.3f}s")
print(f"Sum (sorted):   {sorted_sum}, time: {sorted_time:.3f}s")
print(f"Speedup from sorting: {unsorted_time / sorted_time:.2f}x")

pipeline_depth = 18
prediction_accuracies = [0.70, 0.85, 0.95, 0.99]

print(f"\nMisprediction penalty at {pipeline_depth}-stage pipeline:")
for accuracy in prediction_accuracies:
    penalty = (1 - accuracy) * pipeline_depth
    print(f"  {accuracy:.0%} accuracy → {penalty:.2f} cycles/branch average penalty")

two_bit_states = {"strongly_not_taken": 0, "weakly_not_taken": 1, "weakly_taken": 2, "strongly_taken": 3}
state_names = ["SN", "WN", "WT", "ST"]

branch_outcomes = [True, True, False, True, True, True, False, True]
state = 1
correct = 0

print(f"\n2-bit saturating counter simulation:")
for outcome in branch_outcomes:
    prediction = state >= 2
    hit = prediction == outcome
    correct += hit
    old_state = state
    state = min(3, state + 1) if outcome else max(0, state - 1)
    print(f"  State: {state_names[old_state]} → predict {'T' if prediction else 'N'}, "
          f"actual {'T' if outcome else 'N'}, {'✓' if hit else '✗'} → {state_names[state]}")

print(f"  Accuracy: {correct}/{len(branch_outcomes)} = {correct/len(branch_outcomes):.0%}")

**References:**

[📘 Hennessy, J. & Patterson, D. (2019). *Computer Architecture: A Quantitative Approach (6th ed.).* Morgan Kaufmann.](https://www.elsevier.com/books/computer-architecture/hennessy/978-0-12-811905-1)

[📘 Seznec, A. & Michaud, P. (2006). *A case for (partially) TAgged GEometric history length branch prediction.* JILP.](https://jilp.org/vol8/v8paper1.pdf)

---

[⬅️ Previous: Translation Lookaside Buffer](./02_translation_lookaside_buffer.ipynb) | [Next: SIMD Concepts ➡️](../02_Vectorization/01_single_instruction_multiple_data_concepts.ipynb)